## Neural Network for training CIFAR10 dataset

#### We are going to use:

1. Tensorflow - Open source machine learning library (Offers APIs for developers to use the library to create deep learning models e.g. Keras)
2. Keras - High level neural network API which sits on top of Tensorflow. Developers interact directly with Keras in order to easily use Tensorflow.

Keras is an easy way for users to interact with the deep learning library in python code. It sits on top of libraries (e.g. Tensorflow, Theano) and provides users with understandable feedback enabling faster prototyping.

Reference:
https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py


In [3]:
# Importing libraries 
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# Initializing variables 
batch_size = 1
num_classes = 10
epochs = 1
num_predictions = 20
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
print (model)
print ()

model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
print (model)
print ()


# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



In [8]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
print (model)

prediction = model.predict(x_test)
print (prediction)

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
 1920/50000 [>.............................] - ETA: 8:52 - loss: 2.2662 - acc: 0.1307

KeyboardInterrupt: 

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Sigmoid to prob test

In [1]:
from __future__ import division

import sys
import os
import random
from collections import defaultdict

import h5py
import numpy as np

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from sklearn.externals import joblib
from sklearn import preprocessing

import keras
import keras.backend as K
from keras.regularizers import l2
from keras.models import Model, load_model
from keras.layers import Input, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, Conv1D, MaxPooling2D, UpSampling2D
from keras.layers.core import Lambda
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.layers.merge import Concatenate
from keras.utils import np_utils, plot_model
from keras.callbacks import ReduceLROnPlateau, CSVLogger, ModelCheckpoint, TensorBoard


DROPOUT = 0.1
ACTIVATION = ELU
INIT = "he_normal"

reg_strength = 0.0000000000000000001
REG = l2(reg_strength)



Using TensorFlow backend.


## Functions

In [2]:
def self_outer(x):
    batch_size = K.shape(x)[0]
    outer_x = x[:,:, None,:] * x[:, None,:,:]
    return outer_x


def add_1D_conv(model, filters, kernel_size, padding="same", kernel_initializer=INIT, kernel_regularizer=REG):
    model = Conv1D(filters, kernel_size, padding=padding, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)(model)
    model = ACTIVATION()(model)
    model = BatchNormalization()(model)
    model = Dropout(DROPOUT)(model)
    return model


def add_2D_conv(model, filters, kernel_size, data_format="channels_last", padding="same", depthwise_initializer=INIT, pointwise_initializer=INIT, depthwise_regularizer=REG, pointwise_regularizer=REG, separable=True, namesuffix=""):
    if separable:
        raise ValueError('Separable!')
    if namesuffix:
        model = Conv2D(filters, kernel_size, data_format=data_format, padding=padding, kernel_initializer=depthwise_initializer, kernel_regularizer=depthwise_regularizer, name="separable_conv2d_" + namesuffix)(model)
        model = ACTIVATION(name="activation_" + namesuffix)(model)
        model = BatchNormalization(name="batch_normalization_" + namesuffix)(model)
        model = Dropout(DROPOUT, name="dropout_" + namesuffix)(model)
    else:
        model = Conv2D(filters, kernel_size, data_format=data_format, padding=padding, kernel_initializer=depthwise_initializer, kernel_regularizer=depthwise_regularizer)(model)
        model = ACTIVATION()(model)
        model = BatchNormalization()(model)
        model = Dropout(DROPOUT)(model)
    return model


def add_binary_head(model, dist_cutoff, kernel_size, filters):
    if 'nohead' in modelfile:
        out_reg = l2(0)
    else:
        out_reg = REG
    print(modelfile, out_reg)

    out_binary = Conv2D(1, kernel_size, activation="sigmoid", data_format="channels_last", padding="same", kernel_initializer=INIT,
                        kernel_regularizer=out_reg,
                        name="out_binary_%s" % dist_cutoff)(model)
    return out_binary


def add_2D_block(input_layer, kernel_size=3, filters=16, num_conv=3):
    block = input_layer
    if input_layer.shape[-1] != filters:
        input_layer = add_2D_conv(input_layer, filters, kernel_size)
    for _ in range(num_conv):
        block = add_2D_conv(block, filters, kernel_size)
    res_block = keras.layers.add([input_layer, block])
    return res_block


def create_1D_model(model, kernel_size=3, filters=16, num_conv=3):
    model = add_1D_conv(model, filters, 1)
    model = add_1D_conv(model, filters, kernel_size)
    for _ in range(num_conv - 1):
        model = add_1D_conv(model, filters, kernel_size)
    return model



def wrap_model(model, binary_cutoffs):

    # inputs for sequence features
    inputs_seq = [Input(shape=(None, 22), dtype=K.floatx(), name="seq"), # sequence
                  Input(shape=(None, 23), dtype=K.floatx(), name="self_info"), # self-information
                  Input(shape=(None, 23), dtype=K.floatx(), name="part_entr")] # partial entropy

    # input for 2D features
    #inputs_2D = [Input(shape=(None, None, 1), name="plm", dtype=K.floatx()), # plm
    inputs_2D = [Input(shape=(None, None, 1), name="gdca", dtype=K.floatx()), # gdca
                 Input(shape=(None, None, 1), name="mi_corr", dtype=K.floatx()), # mi_corr
                 Input(shape=(None, None, 1), name="nmi_corr", dtype=K.floatx()), # nmi_corr
                 Input(shape=(None, None, 1), name="cross_h", dtype=K.floatx())] # cross_h

    # input for masking missing residues
    input_mask = Input(shape=(None, None, 1), name="mask")

    out_lst = model(inputs_2D + inputs_seq)
    out_mask_lst = []
    out_names = ["out_sscore_mask"] + ["out_binary_%s_mask" % d for d in binary_cutoffs]

    for i, out in enumerate(out_lst):
        out = keras.layers.Multiply(name=out_names[i])([out, input_mask])
        out_mask_lst.append(out)

    wrapped_model = Model(inputs=inputs_2D + inputs_seq + [input_mask], outputs=out_mask_lst)

    return wrapped_model


def create_unet(filters=64, ss_model_path = "ss_pred_resnet_elu_nolr_dropout01_l26_large_v3_saved_model.h5", binary_cutoffs=[]):

    inputs_seq = [Input(shape=(None, 22), dtype=K.floatx()), # sequence
                  Input(shape=(None, 23), dtype=K.floatx()), # self-information
                  Input(shape=(None, 23), dtype=K.floatx())] # partial entropy

    ss_model = load_model(ss_model_path)
    plot_model(ss_model, "ss_model.png")
    ss_model.trainable = False

    seq_feature_model = ss_model._layers_by_depth[5][0]
    plot_model(seq_feature_model, "seq_feature_model.png")
    assert 'model' in seq_feature_model.name, seq_feature_model.name
    seq_feature_model.name = 'sequence_features'
    seq_feature_model.trainable = False
    for l in ss_model.layers:
        l.trainable = False
    for l in seq_feature_model.layers:
        l.trainable = False

    bottleneck_seq = seq_feature_model(inputs_seq)
    model_1D_outer = Lambda(self_outer)(bottleneck_seq)
    model_1D_outer = BatchNormalization()(model_1D_outer)

    inputs_2D = [Input(shape=(None, None, 1), dtype=K.floatx()), # plm/gdca
                 Input(shape=(None, None, 1), dtype=K.floatx()), # mi_corr
                 Input(shape=(None, None, 1), dtype=K.floatx()), # nmi_corr
                 Input(shape=(None, None, 1), dtype=K.floatx())] # cross_h

    #model_2D_branches = []
    #for i in inputs_2D:
    #    i = add_2D_conv(i, 4, 1)
    #    i = add_2D_conv(i, 4, kernel_size)
    #    i = add_2D_conv(i, 4, kernel_size)
    #    model_2D_branches.append(i)

    #unet = keras.layers.concatenate(model_2D_branches + [model_1D_outer])
    unet = keras.layers.concatenate(inputs_2D + [model_1D_outer])

    unet = add_2D_conv(unet, filters, 1, separable=False)
    unet = add_2D_conv(unet, filters, 3, separable=False)
    unet = add_2D_conv(unet, filters, 3, separable=False)
    link1 = unet
    unet = MaxPooling2D()(unet)
    unet = add_2D_conv(unet, filters*2, 3, separable=False)
    unet = add_2D_conv(unet, filters*2, 3, separable=False)
    link2 = unet
    unet = MaxPooling2D()(unet)
    unet = add_2D_conv(unet, filters*4, 3, separable=False)
    unet = add_2D_conv(unet, filters*4, 3, separable=False)
    link3 = unet
    unet = MaxPooling2D()(unet)
    unet = add_2D_conv(unet, filters*8, 3, separable=False)
    unet = add_2D_conv(unet, filters*8, 3, separable=False)
    link4 = unet
    unet = MaxPooling2D()(unet)
    unet = add_2D_conv(unet, filters*16, 3, separable=False)
    unet = add_2D_conv(unet, filters*16, 3, separable=False)

    unet = UpSampling2D()(unet)
    unet = add_2D_conv(unet, filters*8, 2, separable=False)
    unet = keras.layers.concatenate([unet, link4])
    unet = add_2D_conv(unet, filters*8, 3, separable=False)
    unet = add_2D_conv(unet, filters*8, 3, separable=False)
    unet = UpSampling2D()(unet)
    unet = add_2D_conv(unet, filters*4, 2, separable=False)
    unet = keras.layers.concatenate([unet, link3])
    unet = add_2D_conv(unet, filters*4, 3, separable=False)
    unet = add_2D_conv(unet, filters*4, 3, separable=False)
    unet = UpSampling2D()(unet)
    unet = add_2D_conv(unet, filters*2, 2, separable=False)
    unet = keras.layers.concatenate([unet, link2])
    unet = add_2D_conv(unet, filters*2, 3, separable=False)
    unet = add_2D_conv(unet, filters*2, 3, separable=False)
    unet = UpSampling2D()(unet)
    unet = add_2D_conv(unet, filters, 2, separable=False)
    unet = keras.layers.concatenate([unet, link1])
    split = unet
    unet = add_2D_conv(unet, filters, 3, separable=False)
    unet = add_2D_conv(unet, filters, 3, separable=False)

    out_binary_lst = []
    if binary_cutoffs:
        for d in binary_cutoffs:
            out_binary_lst.append(add_binary_head(unet, d, 7, filters))

    print (out_binary_lst)
    unet = add_2D_conv(split, filters, 3, separable=False)
    unet = add_2D_conv(unet, filters, 3, separable=False)

    if 'nohead' in modelfile:
        out_reg = l2(0)
    else:
        out_reg = REG
    out_sscore = Conv2D(1, 7, activation="sigmoid", data_format="channels_last", padding="same",
                        kernel_initializer=INIT,
                        kernel_regularizer=out_reg,
                        name="out_sscore")(unet)
    print (out_sscore)
    model = Model(inputs=inputs_2D + inputs_seq, outputs=[out_sscore] + out_binary_lst)
    return model


In [3]:

if __name__ == "__main__":
    
    modelfile = 'model_test'
    suffix = os.path.splitext(modelfile)[0]
    #num_blocks = int(sys.argv[2])

    binary_cutoffs = [6, 8, 10]

    model = create_unet(binary_cutoffs=binary_cutoffs)
    #plot_model(model, "unet.png")
    print(model.summary())
    model = wrap_model(model, binary_cutoffs)
    model.save(modelfile)



/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.